# Surviving Titanic

## Introduction and initial question

This time we will take a look at a data set containing passenger data from the Titanic and see if we can find any patterns in the data pointing if any gender, age group, or social class had higher odds of surviving the disastrous shipwreck.

Was it possibly so the the upper class were favored and had a higher survival rate than other passengers? Perhaps age made a difference? And was it really ladies first even for the lifeboats on the sinking ship?

Below we will take a look at and analyze the passenger data to see if we can find the answers to these questions.
However, before we start let's import the libraries we need for our analysis.

In [ ]:
# Importing numpy and pandas for handy data handling. 
import numpy as np
import pandas as pd

# Matplotlib for plotting our data and seaborn's styling for eyecandy.
import matplotlib.pyplot as plt
import seaborn

# We will use statsmodels logistic regression model last to analyze 
# the different probabilities of survival depending on age, gender, and class.  
import statsmodels.api as sm

# Inline plots.
%matplotlib inline

Now we are ready to tackle our choosen data set.

## The dataset

The data set itself is taken from the following [Kaggle competition](https://www.kaggle.com/c/titanic/data) which works as an introduction to machine learning. Today we will use the same data but focus more on our analysis of the data than making predictions.

We start by reading in the data using the pandas CSV reader. 

In [ ]:
passengers = pd.read_csv('titanic-data.csv')
passengers.head(3)

It seems like the data has been read with almost no problem at all, but there are some columns in the given set that needs further explanation. The following explanations are given on the Kaggle competition page.

|Variable|Definition|Key|
|--------|----------|---|
|survival|Survival|0 = No, 1 = Yes|
|pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd|
|sex|Sex||
|Age|Age in years||	
|sibsp|# of siblings / spouses aboard the Titanic||
|parch|# of parents / children aboard the Titanic||
|ticket|Ticket number||
|fare|Passenger fare||
|cabin|Cabin number||
|embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampton|

We can see that there are some 'NaN' values present in the cabin column so just to be sure we will take an extra look at our data with the `.info()` method.

In [ ]:
# Check info about the data we read with .read_csv()
passengers.info()

We can see that in addition to the Cabin, Age, and Embarked columns are also missing values.
We either need to estimate the missing values or drop them. We could estimate missing ages with either median or average values, but for our analysis the non-null 714 values is enough. We choose to just drop the rows with null values.

Cabin has to many missing values and according to wikipedia also a possible bias towards first class passengers. However, something that would be interesting to look up is for example if cabin proximity to the few lifeboats that were available influenced the possibility for survival. This is out of scope for this time so instead we choose to just drop the column instead.

Also, upon further inspection it seems like SibSp/Parch and ticket numbers don't have any much connection making it hard to make sense of which passenger are related to which. There might be some interesting analysis in answering the question if being in a group increased or actually decreased the potential for survival. But, once again this might be a bit to much work for this time so we choose to drop these three columns instead.

The same reasoning goes for the Name, PassengerId, and Fare columns which we also choose to drop.

In [ ]:
# Dropping rows with NaN values
passengers.dropna(subset=['Age'], inplace=True)

# Dropping unneeded columns
passengers.drop('Cabin', axis=1, inplace=True)
passengers.drop('Embarked', axis=1, inplace=True)

passengers.drop('SibSp', axis=1, inplace=True)
passengers.drop('Parch', axis=1, inplace=True)
passengers.drop('Ticket', axis=1, inplace=True)

passengers.drop('Name', axis=1, inplace=True)
passengers.drop('PassengerId', axis=1, inplace=True)
passengers.drop('Fare', axis=1, inplace=True)

Lastly, to make column names more coherent we rename the Pclass column to just Class and to better describe to contents of each column we convert the Sex, Class, and Survived column's data types from Strings to Categories, and Integers to booleans respectively. 

In [ ]:
passengers = passengers.rename(columns={'Pclass': 'Class'})

passengers['Sex'] = passengers.Sex.astype('category')
passengers['Class'] = passengers.Class.astype('category')

passengers['Survived'] = passengers.Survived.apply(bool)

This gives us the below data frame to work with.

In [ ]:
passengers.info()

### My own analysis of the variables types

***NEED TO WRITE***

### Descriptive stats

In [ ]:
passengers['Age'].describe()

In [ ]:
passengers['Sex'].describe()

In [ ]:
passengers['Class'].describe()

In [ ]:
passengers['Survived'].describe()

Based on the above, we can see here that according to our sample the typical passenger was a male around 30 travelling in 3 class and, unfortunately, a casualty from the shipwreck. Worth noting is that after we cleaned up the data we have no missing values in any of our remaining columns.

Lets try to plot some different relationships between class, sex, and age for the survivors and non-survivors in our data set to see if we can find something interesting. We start out with plotting comparisons for the composition of passenger class and sex between survivors and casualties. 

In [ ]:
# Create a grid with two columns for display
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))

# Group passengers by Survived and Class then plot the results in a bar graph
survived_by_class = passengers.groupby(['Survived','Class']).size().unstack()
survived_by_class.plot(kind='bar', stacked=True, ax=axs[0])
axs[0].set_title('1. Survivors by Class')

# Do the same as above but with Survived and Sex
survived_by_sex = passengers.groupby(['Survived', 'Sex']).size().unstack()
survived_by_sex.plot(kind='bar', stacked=True, ax=axs[1])
axs[1].set_title('2. Survivors by Sex')

# Display plots
plt.show()

From the above bar graphs we can see that there is a big change in ratio between male and female, and between 3 class 1/2 class passengers represented in the two different categories (Survived: False/True). It do look as if women had a higher chance of survival than men and 3 class passengers a much lower chance than 1 class passengers. To really make sure that this isn't just in our sample we will perform tests to see if the two deviations are statistically significant or not. 

We also tried plotting the survival rate for passengers born the same year but too much noise did make it hard to get anything out of the plot. By combining the ages in to intervals of 4 we can keep some of the noise down. Judning from the adjusted graph it looks like we might have a correlation between higher age and an increased mortality rate. This will also be a question we would like to answer.

In [ ]:
#Grouped by age in full years
survival_by_age = passengers.groupby(lambda x: int(passengers.loc[x].Age)).Survived
survival_rate_by_age = survival_by_age.apply(lambda x: x[x == True].count()/(x.count() * 1.0))
plt.scatter(survival_rate_by_age.index, survival_rate_by_age);

In [ ]:
#Group by age in intervals of 4
survival_by_age_group = passengers.groupby(lambda x: int(passengers.loc[x].Age/4)*4).Survived
survival_rate_by_age_group = survival_by_age_group.apply(lambda x: x[x == True].count()/(x.count() * 1.0))
plt.scatter(survival_rate_by_age_group.index, survival_rate_by_age_group);

## Chi squared test for independence

After looking at the above three variables and plots we can hypothesize about there being three factors that all had  effect on the chance of survival.

1. Class, lower passenger class also meant lower probability for survival.
2. Sex, women had a higher probability for survival than men.
3. Age, higher age also meant a decreased chance of survival.

We will start by using a [chi squared test for independence](https://en.wikipedia.org/wiki/Chi-squared_test) to test if the frequency of survival for our passengers and dependent on the first two factors, Class and Gender, frequencies.

The test is possible to perform since all our involved variables are categorical, meaning that all our outcomes are mutually exclusive (you cannot be dead and alive, or in first and second class at the same time) and the total probability for all outcomes adds up to 1 (meaning you have to be either be dead or alive etc).

Explained very shortly to do a chi squared test we calculate our chi square statistic, $\chi^2$, as a normalized sum of the squared deviations between observed and theoretical frequencies and then compare it to a critical value (found in a table) based on our confidence level and degrees of freedom. Expressed mathematically it looks something like the below.

$$ \chi^2 = \sum_n{\frac{(O_i - E_i)^2}{E_i}} $$ 

where

$\chi^2 =$ the chi statistic  
$O_i =$ observations of type i  
$E_i = Np_i =$ expected frequency for type i  
$N =$ Total number of observations  
$n =$ number of types of observations  

To test for variable indepence we formulate the following hypothesises.

$H_0$: The variables are independent.  
$H_a$: The variables are **not** independent.

To start our calculations we first define a function for calculating the $\chi^2$-statistic.

Then we define how to perform the chi square test.

In [ ]:
def chi2_test(data, var1, var2):
    '''
    Chi^2 test that checks a defined variable(var2) against another(var1) boolean variable for dependency.
    Returns the chi statistic for the defined variable(var2).
    '''
    var1_by_var2 = pd.DataFrame(data.groupby([var2, var1]).size().unstack()[True])
    rate = data[data[var1] == True][var1].count() / (data[var1].count()*1.0)
    
    var1_by_var2['Total'] = data.groupby([var2]).size()
    var1_by_var2['Expected'] = var1_by_var2['Total'] * survival_rate
    var1_by_var2['Difference'] = var1_by_var2[True] - var1_by_var2['Expected'] 
    var1_by_var2['Difference2'] = var1_by_var2['Difference']**2
    var1_by_var2['Proportion'] = var1_by_var2['Difference2'] / var1_by_var2['Expected']

    return var1_by_var2['Proportion'].sum()

Degrees of freedom is defined as $(rows - 1)(columns - 1)$. In our case with 2 possible outcomes for survival and 3 for class we get $df = 1 * 2 = 2$.  
Further, our $\alpha$(confidence level) for the test is 0.05 which gives us a critical value of 5.99 from our [$\chi^2$table](https://en.wikipedia.org/wiki/Chi-squared_distribution#Table_of_.CF.872_values_vs_p-values).

In [ ]:
chi2_test(passengers, 'Survived', 'Class')

The statistic value is higher than our critical value which means we reject our $H_0$ that the two variables survival and class are independent with a 95% probability.

Similarly for survival and sex we get $df = 1 * 1 = 1$ with two outcomes for both survival and sex.
Using the same $\alpha$ gives us a $\chi^2$ critical value of 3.84.

In [ ]:
chi2_test(passengers, 'Survived', 'Sex')

Once again we get a statistic value larger than our critical value meaning that we once again reject the $H_0$. This means there are a dependency between sex and survival rate with a probability of 95%.

## Logistical regression for probabilities  

Next we will investigate the relation between Age and survival with logistical regression from the statsmodels package.

In [ ]:
# Start by dropping class and sex, for now...
passenger_ages = passengers.drop(['Class', 'Sex'], axis=1)

In [ ]:
logit = sm.Logit(passenger_ages.Survived, passenger_ages.Age)

# Fit the model
result = logit.fit()

result.summary()


As a final step we include variables for Class and Age to see how all three affects the survival rate.

In [ ]:
passenger_dummies = passenger_ages.join(
    pd.get_dummies(passengers['Class'], prefix='Class'))
passenger_dummies = passenger_dummies.join(
    pd.get_dummies(passengers['Sex']))

# To create a baseline, our typical passenger, 
# we drop the class 3 and male column.
passenger_dummies.drop('Class_3', axis=1, inplace=True)
passenger_dummies.drop('male', axis=1, inplace=True)

passenger_dummies.head()

In [ ]:
logit = sm.Logit(passenger_dummies['Survived'], 
                 passenger_dummies[passenger_dummies.columns[1:]])

# Fit the model
result = logit.fit()

result.summary()

In [ ]:
# Last step, visualizing the probability for all possible data points
ages = np.linspace(0, 80, 81, dtype=int)

# define cartesian function
def cartesian(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype
    
    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
        
    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

#enumerate all possible combinations of dummy variables (0=male, 1=female)
combinations = pd.DataFrame(cartesian([ages, [1, 2, 3], [0, 1]]))

#recreate dummies
combinations.columns = ['age', 'class', 'female']

dummy_classes = pd.get_dummies(combinations['class'])
dummy_classes.columns = ['class_1', 'class_2', 'class_3']

dummy_sexes = pd.get_dummies(combinations['female'])
dummy_sexes.columns = ['male', 'female']

#keep what we need for predictions and join with dummies

combinations = pd.DataFrame(combinations['age']).join(dummy_classes.ix[:, :'class_2'])
combinations = combinations.join(dummy_sexes.ix[:, 'female':])

combinations['survival_prediction'] = result.predict(combinations)

# Predicted survival probabilities for a average aged passenger.
combinations[180:186]

In [ ]:
#Pivoting table for plotting
pivot = pd.pivot_table(combinations, 
                       values=['survival_prediction'],
                       index=['age'],
                       columns=['female', 'class_1', 'class_2'])

pivot.head()

In [ ]:
pivot.plot();

### Conclusion